# PRISM-AI API Quickstart Tutorial

This notebook demonstrates basic usage of the PRISM-AI REST API.

## Prerequisites

- Python 3.8+
- requests library
- API key from your administrator

## Installation

```bash
pip install requests numpy pandas matplotlib
```

In [ ]:
import requests
import json
import time
from typing import Dict, Any

# Configuration
API_KEY = "your-api-key-here"  # Replace with your API key
BASE_URL = "http://localhost:8080"  # or production URL

# Headers for all requests
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

print("✓ Configuration loaded")

## 1. Health Check

First, let's verify the API server is running.

In [ ]:
# Basic health check
response = requests.get(f"{BASE_URL}/health")
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

# Check all subsystems
subsystems = ['pwsa', 'finance', 'telecom', 'robotics', 'llm', 'timeseries', 'pixels']

print("\nSubsystem Health:")
for subsystem in subsystems:
    try:
        resp = requests.get(f"{BASE_URL}/api/v1/{subsystem}/health", headers=headers)
        if resp.status_code == 200:
            data = resp.json()['data']
            print(f"  {subsystem:12} ✓ {data['status']}")
        else:
            print(f"  {subsystem:12} ✗ {resp.status_code}")
    except Exception as e:
        print(f"  {subsystem:12} ✗ Error: {e}")

## 2. PWSA Threat Detection

Detect threats from IR sensor data.

In [ ]:
# Sample sensor data
sensor_data = {
    "sv_id": 42,
    "timestamp": int(time.time()),
    "ir_frame": {
        "width": 640,
        "height": 480,
        "centroid_x": 320.0,
        "centroid_y": 240.0,
        "hotspot_count": 5
    }
}

# Detect threat
response = requests.post(
    f"{BASE_URL}/api/v1/pwsa/detect",
    headers=headers,
    json=sensor_data
)

if response.status_code == 200:
    result = response.json()['data']
    print("Threat Detection Result:")
    print(f"  Threat ID: {result['threat_id']}")
    print(f"  Type: {result['threat_type']}")
    print(f"  Confidence: {result['confidence']:.2%}")
    print(f"  Position: {result['position']}")
    print(f"  Time to Impact: {result['time_to_impact']}s")
    print(f"  Action: {result['recommended_action']}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 3. Finance Portfolio Optimization

Optimize asset allocation for a portfolio.

In [ ]:
# Portfolio optimization request
portfolio_request = {
    "assets": [
        {
            "symbol": "AAPL",
            "expected_return": 0.12,
            "volatility": 0.25,
            "current_price": 150.0
        },
        {
            "symbol": "GOOGL",
            "expected_return": 0.15,
            "volatility": 0.30,
            "current_price": 2800.0
        },
        {
            "symbol": "MSFT",
            "expected_return": 0.13,
            "volatility": 0.22,
            "current_price": 380.0
        }
    ],
    "constraints": {
        "max_position_size": 0.5,
        "min_position_size": 0.1,
        "max_total_risk": 0.20
    },
    "objective": "maximize_sharpe"
}

# Optimize
response = requests.post(
    f"{BASE_URL}/api/v1/finance/optimize",
    headers=headers,
    json=portfolio_request
)

if response.status_code == 200:
    result = response.json()['data']
    print("Optimal Portfolio:")
    for weight in result['weights']:
        print(f"  {weight['symbol']:6} {weight['weight']:.1%}")
    print(f"\nExpected Return: {result['expected_return']:.2%}")
    print(f"Expected Risk: {result['expected_risk']:.2%}")
    print(f"Sharpe Ratio: {result['sharpe_ratio']:.2f}")
    print(f"Optimization Time: {result['optimization_time_ms']:.1f}ms")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 4. LLM Query

Query a large language model.

In [ ]:
# Simple LLM query
llm_request = {
    "prompt": "Explain quantum computing in one sentence.",
    "temperature": 0.7,
    "max_tokens": 100
}

response = requests.post(
    f"{BASE_URL}/api/v1/llm/query",
    headers=headers,
    json=llm_request
)

if response.status_code == 200:
    result = response.json()['data']
    print("LLM Response:")
    print(f"  Model: {result['model_used']}")
    print(f"  Response: {result['text']}")
    print(f"  Tokens: {result['tokens_used']}")
    print(f"  Cost: ${result['cost_usd']:.4f}")
    print(f"  Latency: {result['latency_ms']:.0f}ms")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 5. Time Series Forecasting

Forecast future values using ARIMA.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate sample time series
historical_data = [100 + i + np.random.randn() * 2 for i in range(50)]
timestamps = list(range(len(historical_data)))

# Forecast request
forecast_request = {
    "series_id": "sample-series",
    "historical_data": historical_data,
    "timestamps": timestamps,
    "horizon": 20,
    "method": {
        "arima": {
            "p": 2,
            "d": 1,
            "q": 2
        }
    },
    "include_uncertainty": True
}

response = requests.post(
    f"{BASE_URL}/api/v1/timeseries/forecast",
    headers=headers,
    json=forecast_request
)

if response.status_code == 200:
    result = response.json()['data']
    predictions = result['predictions']
    
    print(f"Forecast completed in {result['computation_time_ms']:.1f}ms")
    print(f"MAE: {result['metrics']['mae']:.3f}")
    print(f"RMSE: {result['metrics']['rmse']:.3f}")
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, historical_data, 'b-', label='Historical', linewidth=2)
    forecast_times = list(range(len(historical_data), len(historical_data) + len(predictions)))
    plt.plot(forecast_times, predictions, 'r--', label='Forecast', linewidth=2)
    plt.axvline(x=len(historical_data), color='gray', linestyle=':', alpha=0.7)
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.title('Time Series Forecast')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 6. WebSocket Real-time Updates

Connect to WebSocket for live events.

In [ ]:
# Note: Requires websocket-client library
# pip install websocket-client

import websocket
import threading

def on_message(ws, message):
    data = json.loads(message)
    print(f"Event: {data['type']}")
    if data['type'] == 'ThreatDetected':
        print(f"  Threat ID: {data['threat_id']}")
        print(f"  Confidence: {data['confidence']}")
    elif data['type'] == 'Pong':
        print(f"  Server responded to ping")

def on_error(ws, error):
    print(f"Error: {error}")

def on_close(ws, close_status_code, close_msg):
    print("WebSocket closed")

def on_open(ws):
    print("WebSocket connected")
    # Send a ping
    ws.send(json.dumps({"type": "Ping"}))

# Connect
ws_url = BASE_URL.replace('http://', 'ws://').replace('https://', 'wss://')
ws = websocket.WebSocketApp(
    f"{ws_url}/ws",
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

# Run in background
wst = threading.Thread(target=ws.run_forever)
wst.daemon = True
wst.start()

print("WebSocket running in background...")
print("Run this cell and then trigger events via API calls")

# To close: ws.close()

## Summary

This tutorial covered:

1. ✓ Health checks
2. ✓ PWSA threat detection
3. ✓ Finance portfolio optimization
4. ✓ LLM queries
5. ✓ Time series forecasting
6. ✓ WebSocket streaming

## Next Steps

- See `02_pwsa_advanced.ipynb` for advanced PWSA features
- See `03_llm_consensus.ipynb` for multi-model consensus
- See `04_pixel_processing.ipynb` for IR frame analysis
- Check [API Documentation](../docs/API.md) for all endpoints